In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# --- 1. T-Net Architecture (PyTorch Module) ---
class TNet(nn.Module):
    """
    PointNet's T-Net (Transformation Network) for learning 3x3 rotation matrices.
    It takes a point cloud as input and outputs 6 numbers that are then
    orthogonalized to form an SO(3) rotation matrix.
    """
    def __init__(self, k=3):
        """
        Initializes the T-Net.
        Args:
            k (int): The dimension of the transformation matrix (e.g., 3 for 3x3 spatial transform).
                     The output will be k*k, but we predict 2 vectors of size k.
        """
        super(TNet, self).__init__()
        self.k = k

        # Shared MLPs (Multi-Layer Perceptrons)
        # These layers process each point independently.
        self.mlp1 = nn.Sequential(
            nn.Conv1d(k, 128, 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 512, 1),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Conv1d(512, 2048, 1),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
        )

        # Fully Connected Layers after Max Pooling
        # These layers process the global feature vector to predict the transformation.
        # Output is 2*k (6 for 3x3 matrix, representing two 3D vectors)
        self.fc = nn.Sequential(
            nn.Linear(2048, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 2 * k), # Output 6 numbers for 3x3 matrix (two 3D vectors)
        )

    def forward(self, x):
        """
        Forward pass of the T-Net.
        Args:
            x (torch.Tensor): Input point cloud tensor of shape (B, N, k),
                              where B is batch size, N is number of points, k is dimension (e.g., 3 for XYZ).
        Returns:
            torch.Tensor: A tensor of shape (B, 2*k) representing the raw output
                          before orthogonalization.
        """
        # Permute input to (B, k, N) for Conv1d
        x = x.permute(0, 2, 1)

        # Apply shared MLPs
        x = self.mlp1(x) # Output shape: (B, 1024, N)

        # Max pooling across the N points to get a global feature vector
        x = torch.max(x, 2, keepdim=True)[0] # Output shape: (B, 1024, 1)
        x = x.view(-1, 2048) # Reshape to (B, 1024)

        # Apply fully connected layers to predict the 6 numbers
        x = self.fc(x) # Output shape: (B, 2*k)

        return x

# --- 2. Differentiable Gram-Schmidt Orthogonalization ---
def gram_schmidt_orthogonalization(vectors_6d):
    """
    Performs differentiable Gram-Schmidt orthogonalization on a batch of 6D vectors
    to produce a batch of 3x3 SO(3) rotation matrices.

    Args:
        vectors_6d (torch.Tensor): A tensor of shape (B, 6), where B is batch size.
                                   Each 6D vector represents two 3D vectors [v1_x, v1_y, v1_z, v2_x, v2_y, v2_z].

    Returns:
        torch.Tensor: A tensor of shape (B, 3, 3) representing the batch of
                      SO(3) rotation matrices.
    """
    batch_size = vectors_6d.shape[0]

    # Reshape the 6D output into two 3D vectors for each item in the batch
    v1 = vectors_6d[:, :3] # (B, 3)
    v2 = vectors_6d[:, 3:] # (B, 3)

    # Step 1: Normalize v1 to get u1
    # Add a small epsilon for numerical stability to avoid division by zero if magnitude is 0
    u1 = v1 / (torch.norm(v1, dim=1, keepdim=True) + 1e-8)

    # Step 2: Project v2 onto u1, then subtract from v2 to get u2_prime
    # dot product: (v2 * u1).sum(dim=1, keepdim=True) -> (B, 1)
    proj_u1_v2 = (u1 * v2).sum(dim=1, keepdim=True) * u1 # (B, 3)
    u2_prime = v2 - proj_u1_v2 # (B, 3)

    # Step 3: Normalize u2_prime to get u2
    # Handle cases where u2_prime might be very small or zero (e.g., v1 and v2 are collinear)
    # In such cases, we need to find an orthogonal vector.
    # A robust way is to use cross product with a default vector, then re-orthogonalize.
    # For training, the network should learn to avoid degenerate cases.
    u2 = u2_prime / (torch.norm(u2_prime, dim=1, keepdim=True) + 1e-8)

    # Step 4: Calculate u3 as the cross product of u1 and u2
    u3 = torch.cross(u1, u2, dim=1) # (B, 3)
    u3 = u3 / (torch.norm(u3, dim=1, keepdim=True) + 1e-8) # Re-normalize for numerical stability

    # Construct the rotation matrix by stacking u1, u2, u3 as columns
    # R = [u1.T, u2.T, u3.T]
    # u1, u2, u3 are (B, 3). Stack them along a new dimension, then permute.
    rotation_matrix = torch.stack([u1, u2, u3], dim=2) # (B, 3, 3)

    return rotation_matrix

# --- 3. Geodesic Angular Distance Loss ---
def geodesic_loss(R_pred, R_true, L2=True, reduction='mean'):
    """
    Calculates the geodesic angular distance between two batches of rotation matrices.
    The formula is: theta = arccos((trace(R_pred^T * R_true) - 1) / 2)
    Args:
        R_pred (torch.Tensor): Predicted rotation matrices (B, 3, 3).
        R_true (torch.Tensor): Ground truth rotation matrices (B, 3, 3).
    Returns:
        torch.Tensor: Mean geodesic angular distance in radians.
    """
    # R_pred_T * R_true
    R_diff = torch.matmul(R_pred.transpose(1, 2), R_true) # (B, 3, 3)

    # Trace of R_diff
    # Sum diagonal elements: R_diff[:, 0, 0] + R_diff[:, 1, 1] + R_diff[:, 2, 2]
    trace = R_diff[:, 0, 0] + R_diff[:, 1, 1] + R_diff[:, 2, 2] # (B,)

    # Clamp the argument to arccos to avoid NaN due to floating point errors
    # The argument should be between -1 and 3 for rotation matrices, but numerical
    # inaccuracies can push it slightly outside [-1, 3].
    # For arccos((trace - 1)/2), the argument should be in [-1, 1].
    # trace is in [-1, 3] for valid rotation matrices.
    # (trace - 1) / 2 is in [-1, 1].
    arg = torch.clamp((trace - 1) / 2, -1.0 + 1e-7, 1.0 - 1e-7) # Add epsilon for robustness

    # Calculate angle in radians
    angle = torch.acos(arg) # (B,)
    if(L2):
        angle = angle*angle
    if(reduction != 'mean'):
        return angle
    # Return mean angle across the batch
    return torch.mean(angle)

def trace_loss(R_pred, R_true):
    R_diff = torch.matmul(R_pred.transpose(1, 2), R_true)
    trace = R_diff[:, 0, 0] + R_diff[:, 1, 1] + R_diff[:, 2, 2]
    return -torch.mean(trace)
def point_alignment_loss(P_aligned, P_canonical):
    # P_aligned, P_canonical: (B, N, 3)
    return torch.mean(torch.norm(P_aligned - P_canonical, dim=2))  # scalar


In [2]:
from scipy.spatial.transform import Rotation as R # For robust Euler angle conversion
from torch.utils.data import Dataset
import os
import numpy as np
from sklearn.decomposition import PCA

from scipy.spatial.transform import Rotation as R_scipy
from scipy.spatial.transform import Rotation as R_scipy # Import scipy's Rotation class
class PointCloudDataset(Dataset):
    def __init__(self, npz_path, npz_data_key='data',dataidx = None):
        """
        Initializes the dataset.
        Args:
            npz_path (str): Path to the .npz file containing canonical point clouds.
            npz_data_key (str): The key to access the point cloud data within the .npz file.
        """
        if not os.path.exists(npz_path):
            raise FileNotFoundError(f"NPZ file not found at: {npz_path}")

        loaded_data = np.load(npz_path)
        self.canonical_point_clouds = loaded_data[npz_data_key][dataidx]
        self.canonical_point_clouds = self.canonical_point_clouds[:,:2000,:]
        if self.canonical_point_clouds.dtype != np.float32:
            self.canonical_point_clouds = self.canonical_point_clouds.astype(np.float32)
        #print(np.mean(self.canonical_point_clouds.reshape(-1,3), axis =0))
        #print(np.std(self.canonical_point_clouds.reshape(-1,3), axis =0))
        #self.std_point_clouds = np.array([0.2584901 , 0.1014245,  0.15653647])[None,:]
        #self.canonical_point_clouds /= std_point_clouds #mean is near 0,0,0

        print(f"Dataset loaded: {self.canonical_point_clouds.shape} from {npz_path} under key '{npz_data_key}'")

    def __len__(self):
        """Returns the total number of shapes in the dataset."""
        return self.canonical_point_clouds.shape[0]



    def __getitem__(self, idx):
      """
      Returns:
          aligned_pc (torch.Tensor): Point cloud (N, 3) aligned to XYZ via PCA + flips.
          gt_rotation (torch.Tensor): The inverse of PCA alignment (rotation to bring back to original rotated_pc).
      """
      # Step 1: Canonical point cloud
      canonical_pc = self.canonical_point_clouds[idx]  # (N, 3), numpy

      # Step 2: Apply random rotation
      R_rand_np = R_scipy.random().as_matrix()         # (3, 3)
      pc_rotated = canonical_pc @ R_rand_np.T          # Shape: (N, 3)
        
      #pc_rotated /= self.std_point_clouds

      # Step 3: PCA on rotated point cloud
      #pca = PCA(n_components=3)
      #pca.fit(pc_rotated)
      #R_pca_np = pca.components_.T                     # (3, 3), PCA basis vectors as columns

      # Step 4: Flip PCA axes randomly to simulate ambiguity
      #flip = torch.randint(0, 2, (3,)) * 2 - 1          # {+1, -1}
      #F_np = np.diag(flip.numpy())                     # (3, 3) flip matrix
      #R_pca_flipped_np = R_pca_np @ F_np               # Apply sign flip to PCA axes

      # Step 5: Align the point cloud to XYZ axes
      #aligned_pc_np = pc_rotated @ R_pca_flipped_np    # Now aligned to standard axes

      # Step 6: Ground truth is inverse of PCA alignment
      # If aligned_pc = pc_rotated @ R_pca_flipped, then pc_rotated = aligned_pc @ R_pca_flipped.T
      # So to undo all the transformation and recover canonical_pc:
      # canonical = aligned_pc @ R_pca_flipped.T @ R_rand
      # Ground truth = R_pca_flipped.T @ R_rand
      gt_rotation_np = R_rand_np  # Final inverse transform

      # Convert to torch
      pc_rotated = torch.from_numpy(pc_rotated).float()
      gt_rotation = torch.from_numpy(gt_rotation_np).float()

      return torch.from_numpy(canonical_pc).float(), pc_rotated, gt_rotation


In [3]:
# --- DataLoader Setup ---
BATCH_SIZE = 144
NUM_WORKERS = 8 # Adjust based on your system's CPU cores and memory
                # Set to 0 if you encounter issues, then increase slowly.

NPZ_FILE_PATH = "PointCloudSample.npz"
NPZ_DATA_KEY = "pointcloud"
# Instantiate your dataset
idx = np.arange(12000)
trainidx = np.load("TrainIndexPairs.npy")
trainidx = trainidx[np.where(trainidx < 12000)]
# Find the elements in idx that are not in trainidx
validx = np.setdiff1d(idx, trainidx)

traindataset = PointCloudDataset(npz_path=NPZ_FILE_PATH, npz_data_key=NPZ_DATA_KEY, dataidx=trainidx)
valdataset = PointCloudDataset(npz_path=NPZ_FILE_PATH, npz_data_key=NPZ_DATA_KEY, dataidx=validx)
from torch.utils.data import DataLoader
# Instantiate your DataLoader
dataloader = DataLoader(
    traindataset,
    batch_size=BATCH_SIZE,
    shuffle=True, # Shuffle for training
    num_workers=NUM_WORKERS,
    pin_memory=True # For faster data transfer to GPU
)
valloader =  DataLoader(
    valdataset,
    batch_size=BATCH_SIZE,
    shuffle=True, # Shuffle for training
    num_workers=NUM_WORKERS,
    pin_memory=True # For faster data transfer to GPU
)
print(f"\nDataLoader created with Batch Size: {BATCH_SIZE}, Num Workers: {NUM_WORKERS}")
print(f"Total batches per epoch: {len(dataloader)}")

Dataset loaded: (11699, 2000, 3) from PointCloudSample.npz under key 'pointcloud'
Dataset loaded: (301, 2000, 3) from PointCloudSample.npz under key 'pointcloud'

DataLoader created with Batch Size: 144, Num Workers: 8
Total batches per epoch: 82


In [4]:
import torch.optim as optim # Ensure optim is imported
from tqdm import tqdm
# --- Training Loop Example ---
lambdas = 2
#std_point_clouds = torch.from_numpy(np.array([0.2584901 , 0.1014245,  0.15653647])[None,:]).cuda().float()
if __name__ == "__main__":
    # Ensure a device is chosen (GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Initialize T-Net model
    # The T-Net operates on 3-dim points (k=3)
    tnet_model = TNet(k=3).to(device)
    
    from torch.optim.lr_scheduler import CosineAnnealingLR
    NUM_EPOCHS = 900 # Set a small number for demonstration

    optimizer = optim.Adam(tnet_model.parameters(), lr=0.001, fused=True)

    # Create cosine annealing scheduler for 50 epochs
    scheduler = CosineAnnealingLR(
        optimizer,
        T_max=NUM_EPOCHS,          # Total number of epochs
        eta_min=1e-5      # Final learning rate
    )
    

    print("\n--- Starting Training Loop Demonstration ---")
    batch = 0
    best_loss = 100
    val_epoch = 5
    accum_steps = 2  # Number of batches to accumulate
    batch = 0

    for epoch in range(NUM_EPOCHS):
        tnet_model.train()
        total_loss = 0.0

        optimizer.zero_grad()  # Reset before accumulation loop

        for i, (canonical_pc_tensor, point_clouds_batch, gt_rotations_batch) in enumerate(tqdm(dataloader)):
            point_clouds_batch = point_clouds_batch.to(device)
            gt_rotations_batch = gt_rotations_batch.to(device)
            canonical_pc_tensor = canonical_pc_tensor.to(device)

            batch += 1

            raw_6d_output = tnet_model(point_clouds_batch)
            predicted_rotations = gram_schmidt_orthogonalization(raw_6d_output)
            P_aligned = torch.bmm(point_clouds_batch, predicted_rotations)

            loss_geod = geodesic_loss(predicted_rotations, gt_rotations_batch, L2=False)
            point_loss = lambdas * point_alignment_loss(P_aligned, canonical_pc_tensor)
            loss = (loss_geod + point_loss) / accum_steps  # Normalize loss

            loss.backward()  # Accumulate gradients

            if (i + 1) % accum_steps == 0 or (i + 1) == len(dataloader):
                optimizer.step()
                optimizer.zero_grad()

            total_loss += loss_geod.item()  # Only count the geodesic loss for averaging

            if batch % 100 == 0:
                print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Batch {batch}, Geod Loss: {loss_geod.item():.6f}, Point Loss: {point_loss.item():.6f}")

        avg_loss = total_loss / len(dataloader)

        print(f"Epoch {epoch+1} finished. Average Loss: {avg_loss:.4f} radians ({np.degrees(avg_loss):.2f} degrees)")
        if(epoch % val_epoch == 0):


            tnet_model.eval()
            batch = 0
            total_loss = 0
            total_point_loss = 0
            allloss = []

            for (canonical_pc_tensor,point_clouds_batch, gt_rotations_batch) in tqdm(valloader):
                # Move data to the appropriate device (CPU/GPU)
                #batch = 0
                #total_loss = 0
                with torch.no_grad():

                  point_clouds_batch = point_clouds_batch.to(device) # Shape: (B, 8000, 3)
                  gt_rotations_batch = gt_rotations_batch.to(device)   # Shape: (B, 3, 3)
                  canonical_pc_tensor= canonical_pc_tensor.to(device)

                  # Zero the parameter gradients
                  #optimizer.zero_grad()
                  batch +=1

                  # Forward pass: Get the 6D output from T-Net
                  raw_6d_output = tnet_model(point_clouds_batch) # Shape: (B, 6)

                  # Convert 6D output to 3x3 rotation matrix using Gram-Schmidt
                  predicted_rotations = gram_schmidt_orthogonalization(raw_6d_output) # Shape: (B, 3, 3)
                  #point_clouds_batch =point_clouds_batch * std_point_clouds
                  # Calculate geodesic loss
                  geodloss = geodesic_loss(predicted_rotations, gt_rotations_batch, L2=False,reduction=None)
                  P_aligned = torch.bmm(point_clouds_batch, predicted_rotations)  # (B, N, 3)
                  point_loss = point_alignment_loss(P_aligned, canonical_pc_tensor)


                  # Backward pass and optimize


                  total_loss += geodloss.mean().item()
                  allloss.extend(geodloss.flatten().detach().cpu().numpy())
                  total_point_loss += point_loss.item()
            avg_loss = total_loss / len(valloader)
            avg_point = total_point_loss / len(valloader)
            std_loss = np.std(np.array(allloss))
            if(avg_loss < best_loss):
                print("Best loss improved")
                best_loss = avg_loss
                torch.save(tnet_model.state_dict(), f"tnet_model_weights_{avg_loss}_{avg_point}_std_{std_loss}.pth")
            print(f"Epoch {epoch+1} finished. Average val Loss: {avg_loss:.4f} radians ({np.degrees(avg_loss):.2f} degrees), Point loss : {avg_point:.4f}")
            print(f"Std of geodesic loss : {std_loss:.4f} radians ({np.degrees(std_loss):.2f} degrees)")
        scheduler.step()


    print("\nTraining demonstration complete!")

    # Clean up the dummy npz file
    # os.remove(NPZ_FILE_PATH)
    # print(f"Cleaned up dummy file: {NPZ_FILE_PATH}")

Using device: cuda

--- Starting Training Loop Demonstration ---


100%|██████████| 82/82 [00:19<00:00,  4.26it/s]


Epoch 1 finished. Average Loss: 1.4491 radians (83.03 degrees)


100%|██████████| 3/3 [00:00<00:00,  5.03it/s]


Best loss improved
Epoch 1 finished. Average val Loss: 1.3212 radians (75.70 degrees), Point loss : 0.2080
Std of geodesic loss : 0.7196 radians (41.23 degrees)


...

Epoch 899 finished. Average Loss: 0.2824 radians (16.18 degrees)


 62%|██████▏   | 51/82 [00:12<00:07,  4.37it/s]

Epoch 900/900, Batch 300, Geod Loss: 0.279678, Point Loss: 0.128719


100%|██████████| 82/82 [00:19<00:00,  4.19it/s]

Epoch 900 finished. Average Loss: 0.2816 radians (16.14 degrees)

Training demonstration complete!
